## Download data and source folder (**Do not Modify**)


In [ ]:
!pip install gdown
import gdown
import zipfile
url = 'https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi'
output = 'source.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile(output, "r") as zip_ref:
    zip_ref.extractall('.')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1CQdgTOUlY-TUoWZyxtVZxRthBhSuhDVi
To: /content/source.zip
100%|██████████| 11.8M/11.8M [00:00<00:00, 71.4MB/s]


## Install package for calculating FLOPS (**Do not Modify**)


In [ ]:
!pip install pthflops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Necessary Dependencies
- You may import other packages if you want.


In [ ]:
import os
import sys
import os.path as osp

from PIL import Image

import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

import glob
import matplotlib.pyplot as plt

from pthflops import count_ops

# functions that you downloaded from the first code cell.
# please use this code for seed reset, dataloaders, and test function  
from src.util import reset
# if data.py is changed, please submit that also.
from src.data import get_dataloader
from src.test import test

## Define label names, data directory, device name.


In [ ]:
## DO NOT MODIFY FROM HERE
label_names = ['bug', 'electric', 'fighting', 'fire', 'flying', 'grass', 'ground', 'phychic', 'poison', 'water']
data_dir = 'data'
## DO NOT MODIFY UNTIL HERE

## You may modify device name depending on your workspace spec.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Specify the batch size as you want.
# since the epoch is too small, increasing the number of gradient updates by using small batch size was the key point of this assignment.
batch_size = 1

## **(TO-DO)** DEFINE YOUR `MyModel`
- Please do not change the class name. Let `MyModel` be your classifier class name. 
- Below is just simple example using ResNet18.

In [ ]:
class MyModel(nn.Module):
    def __init__(self, dim_hidden=32, dim_output = len(label_names)):
        super().__init__()

        self.conv1 = nn.Conv2d(3, dim_hidden, 5, 2, 2)
        self.conv2 = nn.Conv2d(dim_hidden, dim_hidden, 5, 2, 2)
        self.conv3 = nn.Conv2d(dim_hidden, dim_hidden, 5, 2, 2)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.adapt_pool = nn.AdaptiveAvgPool2d(1)
        self.acti = nn.GELU()

        self.fc = nn.Linear(dim_hidden, dim_output)

    def forward(self, inp):
        B, C, H, W = inp.shape

        out = self.pool(self.acti(self.conv1(inp)))
        out = self.pool(self.acti(self.conv2(out)))
        out = self.pool(self.acti(self.conv3(out)))

        out = self.adapt_pool(out)
        out = out.view(B, -1)

        return self.fc(out)


## **(Optional)** Make your own loss function.
- You may change `criterion` as you want. 
- But make your custom loss function work without changing below lines, which starts from `model.train()`


In [ ]:
def train(model, optimizer, sample):
    ### You may define your own loss function.###
    # weight = [0 for _ in range(len(label_names))]
    # for l in range(len(label_names)):
    #     weight[l] = 1-sum(sample['label']==l)/len(sample['label'])
    # weight = torch.Tensor(weight).to(device)

    criterion = nn.CrossEntropyLoss()
    #############################################

    ### make your code work without changing below lines ###
    model.train()    
    
    input = sample['img'].float().to(device)
    label = sample['label'].long().to(device)

    pred = model(input)

    loss = criterion(pred, label)

    num_correct = torch.sum(torch.argmax(pred, dim=-1)==label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), num_correct.item()
    ##########################################################

## **(TO-DO)** DEFINE YOUR `get_optimizer`
- Please do not change the function name. Let `get_optimizer` be your classifier class name. 
- This function should return the optimizer for optimizing model parameters properly


In [ ]:
######## Define your own function for optimizer.  #######
def get_optimizer(model, lr=1e-4, wd=1e-7):
    return optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
##########################################################

## Repeat Training with different 10 random seeds.

*   Do not change `max_epoch` and `num_seeds`.
*   You  may change lines for `MyModel` and `get_optimizer` part if they require additional inputs.


In [ ]:
### Do not change below parameters ###
max_epoch = 5
num_seeds = 10
### Do not change above parameters ###


### Make your code work without changing below lines except for "optimizer" part ###
total_best_accu = []

for seed in range(num_seeds):
    # do not erase this random seed initialization part
    reset(seed)

    # get dataloader by spliting train/test data randomly.
    train_loader, test_loader, train_dataset, test_dataset = get_dataloader(data_dir, label_names, batch_size)

    # you may change below line for model definition if your "MyModel" requires more inputs.
    model = MyModel().to(device)

    # you may change below line for optimizer if your "get optimizer" requires more inputs.
    optimizer = get_optimizer(model)

    ###### do not change below lines. Make your code work with below lines. ####
    best_accu = -100
    for epoch in range(max_epoch):
        avg_tr_loss = 0.0
        avg_tr_correct = 0.0
        for sample in train_loader:
            tr_loss, tr_correct = train(model, optimizer, sample)

            avg_tr_loss += tr_loss / len(train_loader)
            avg_tr_correct += tr_correct / len(train_dataset)

        avg_te_correct = 0.0
        for sample in test_loader:
            te_correct = test(model, sample, device)
            avg_te_correct += te_correct / len(test_dataset)
    
        best_accu = max(avg_te_correct, best_accu)

    print('<<<<<[SEED {}] BEST ACCU : {}>>>>>'.format(seed, best_accu))    
    total_best_accu.append(best_accu)
    if seed < num_seeds-1:
        del model, optimizer
    ############################################################################   


<<<<<[SEED 0] BEST ACCU : 0.5108433734939773>>>>>
<<<<<[SEED 1] BEST ACCU : 0.4457831325301217>>>>>
<<<<<[SEED 2] BEST ACCU : 0.50120481927711>>>>>
<<<<<[SEED 3] BEST ACCU : 0.47469879518072433>>>>>
<<<<<[SEED 4] BEST ACCU : 0.5180722891566278>>>>>
<<<<<[SEED 5] BEST ACCU : 0.4963855421686763>>>>>
<<<<<[SEED 6] BEST ACCU : 0.5108433734939773>>>>>
<<<<<[SEED 7] BEST ACCU : 0.49156626506024254>>>>>
<<<<<[SEED 8] BEST ACCU : 0.48915662650602565>>>>>
<<<<<[SEED 9] BEST ACCU : 0.49156626506024254>>>>>


## Calculate your grade **(Do not Modify)**
- Grade is calculated based on your `average best accuracy` and `FLOPS`.
- Higher accuracy and lower flops make your grade better.

In [ ]:
###### do not change below lines #####
model = MyModel().to(device)
num_ops = count_ops(model, torch.rand(1, 3, 112, 112).to(device), verbose=False)

mean_accu = np.mean(total_best_accu)
accu_thres = 0.75
accu_point = min(1, np.exp(-2*(accu_thres-mean_accu)))

flops = num_ops[0]
flops_thres = 2e8
flop_disadvantage = flops_thres / (flops_thres + flops)

print('*'*50)
print('Mean Accuracy :', mean_accu)
print('Accuracy Point:', accu_point)
print('*'*50)
print('Flops: ', flops)
print('Flops Advantage: ', flop_disadvantage)
print('*'*50)

point = accu_point * flop_disadvantage
print('Total Points : ', point)

threshold = 0.5
max_point = 50

if point > threshold:
    grade = max_point
else:
    grade = np.exp(-2*(threshold - point)) * max_point

print('YOUR grade is {} point'.format(grade))

Input size: (1, 3, 112, 112)
13,141,546 FLOPs or approx. 0.01 GFLOPs
**************************************************
Mean Accuracy : 0.49301204819277267
Accuracy Point: 0.5981128062458988
**************************************************
Flops:  13141546
Flops Advantage:  0.9383435738051745
**************************************************
Total Points :  0.5612353081514186
YOUR grade is 50 point
